In [1]:
import pandas as pd
import numpy as np
from astroquery.mast import Catalogs
from astropy.coordinates import SkyCoord
import astropy.units as u

In [16]:
def load_membership_list(file_path):
    """Load the membership list from a file."""
    return pd.read_csv(file_path)

def query_tic(ra, dec):
    """Query the TIC catalog for a given RA and Dec."""
    coord = SkyCoord(ra=ra*u.degree, dec=dec*u.degree)
    result = Catalogs.query_region(coord, radius=0.001*u.deg, catalog="TIC")
    return result

# Sample NGC 6819 membership list with KIC IDs (replace with actual data)
ngc6819_members = pd.DataFrame({
    'KIC': [5111207, 5023899, 5024227, 5023760, 5024122, 5113601, 5112499, 5026583, 4938993, 5111834,
            5111908, 5024856, 5112507, 5024280, 5023796, 5024008, 5023724, 5023875, 5112268, 5111939,
            5025271, 4937169, 5112871, 5023666, 5024182, 5023926, 4937149, 4936891, 4937119, 4937356],
    'RA': [295.28458, 295.46213, 295.45978, 295.32355, 295.31873, 295.32724, 295.46078, 295.21735, 
           295.27377, 295.28732, 295.31842, 295.45093, 295.38077, 295.32724, 295.48907, 295.18051, 
           295.42371, 295.35238, 295.32745, 295.30096, 295.30096, 295.42371, 295.35238, 295.32724,
           295.32745, 295.30096, 295.42371, 295.35238, 295.32724, 295.32745],
    'DEC': [40.21639, 40.18481, 40.19511, 40.20554, 40.21249, 40.21387, 40.20777, 40.20320, 
            40.20909, 40.17926, 40.18683, 40.22015, 40.21639, 40.24905, 40.18657, 40.26066, 
            40.19649, 40.20320, 40.18131, 40.18915, 40.18915, 40.19649, 40.20320, 40.21387,
            40.18131, 40.18915, 40.19649, 40.20320, 40.21387, 40.18131],
    'V': [14.787, 14.712, 14.718, 14.698, 14.892, 15.043, 15.102, 15.393, 15.678, 15.384,
          15.344, 15.671, 15.495, 15.738, 15.560, 15.784, 15.783, 15.880, 15.834, 15.945,
          15.965, 16.432, 16.516, 16.515, 16.604, 16.677, 16.929, 17.033, 17.134, 17.266]
})

def query_tic(ra, dec):
    """Query the TIC catalog for a given RA and Dec."""
    coord = SkyCoord(ra=ra*u.degree, dec=dec*u.degree)
    result = Catalogs.query_region(coord, radius=0.002*u.deg, catalog="TIC")
    return result

def check_tess_observability(tic_result, kic_vmag):
    """Check if a target is observable by TESS based on magnitude and contamination."""
    if len(tic_result) == 0:
        return False
    
    # TESS magnitude limit (adjust as needed)
    mag_limit = 15
    
    # Use V magnitude if Tmag is not available
    tmag = tic_result['Tmag'][0] if 'Tmag' in tic_result.columns else kic_vmag
    
    if tmag > mag_limit:
        return False
    
    # Check contamination ratio (if available)
    if 'contratio' in tic_result.columns:
        contamination = tic_result['contratio'][0]
        if contamination > 0.5:  # Arbitrary threshold, adjust as needed
            return False
    
    return True

In [17]:
total_stars = len(ngc6819_members)
tic_matches = 0
tess_observable = 0

for _, star in ngc6819_members.iterrows():
    tic_result = query_tic(star['RA'], star['DEC'])
    
    if len(tic_result) > 0:
        tic_matches += 1
        
        if check_tess_observability(tic_result, star['V']):
            tess_observable += 1

print(f"Total KIC stars: {total_stars}")
print(f"KIC stars with TIC matches: {tic_matches}")
print(f"KIC stars potentially observable by TESS: {tess_observable}")

# Print details of all stars
print("\nDetails of all NGC 6819 stars:")
for _, star in ngc6819_members.iterrows():
    tic_result = query_tic(star['RA'], star['DEC'])
    tic_id = tic_result['ID'][0] if len(tic_result) > 0 else "No match"
    observable = "Yes" if check_tess_observability(tic_result, star['V']) else "No"
    print(f"KIC: {star['KIC']}, RA: {star['RA']:.5f}, DEC: {star['DEC']:.5f}, V: {star['V']:.3f}, TIC: {tic_id}, TESS Observable: {observable}")

Total KIC stars: 30
KIC stars with TIC matches: 27
KIC stars potentially observable by TESS: 2

Details of all NGC 6819 stars:
KIC: 5111207.0, RA: 295.28458, DEC: 40.21639, V: 14.787, TIC: 1880468872, TESS Observable: No
KIC: 5023899.0, RA: 295.46213, DEC: 40.18481, V: 14.712, TIC: 184010868, TESS Observable: No
KIC: 5024227.0, RA: 295.45978, DEC: 40.19511, V: 14.718, TIC: No match, TESS Observable: No
KIC: 5023760.0, RA: 295.32355, DEC: 40.20554, V: 14.698, TIC: 139154018, TESS Observable: No
KIC: 5024122.0, RA: 295.31873, DEC: 40.21249, V: 14.892, TIC: 1880383600, TESS Observable: No
KIC: 5113601.0, RA: 295.32724, DEC: 40.21387, V: 15.043, TIC: 139154116, TESS Observable: No
KIC: 5112499.0, RA: 295.46078, DEC: 40.20777, V: 15.102, TIC: 184010758, TESS Observable: No
KIC: 5026583.0, RA: 295.21735, DEC: 40.20320, V: 15.393, TIC: 1880468759, TESS Observable: No
KIC: 4938993.0, RA: 295.27377, DEC: 40.20909, V: 15.678, TIC: 139154051, TESS Observable: Yes
KIC: 5111834.0, RA: 295.28732, DE

In [19]:
tic_result.columns

<TableColumns names=('ID','ra','dec','pmRA','pmDEC','Tmag','objType','typeSrc','version','HIP','TYC','UCAC','TWOMASS','SDSS','ALLWISE','GAIA','APASS','KIC','POSflag','e_pmRA','e_pmDEC','PMflag','plx','e_plx','PARflag','gallong','gallat','eclong','eclat','Bmag','e_Bmag','Vmag','e_Vmag','umag','e_umag','gmag','e_gmag','rmag','e_rmag','imag','e_imag','zmag','e_zmag','Jmag','e_Jmag','Hmag','e_Hmag','Kmag','e_Kmag','TWOMflag','prox','w1mag','e_w1mag','w2mag','e_w2mag','w3mag','e_w3mag','w4mag','e_w4mag','GAIAmag','e_GAIAmag','e_Tmag','TESSflag','SPFlag','Teff','e_Teff','logg','e_logg','MH','e_MH','rad','e_rad','mass','e_mass','rho','e_rho','lumclass','lum','e_lum','d','e_d','ebv','e_ebv','numcont','contratio','disposition','duplicate_id','priority','eneg_EBV','epos_EBV','EBVflag','eneg_Mass','epos_Mass','eneg_Rad','epos_Rad','eneg_rho','epos_rho','eneg_logg','epos_logg','eneg_lum','epos_lum','eneg_dist','epos_dist','distflag','eneg_Teff','epos_Teff','TeffFlag','gaiabp','e_gaiabp','gaiarp','

In [30]:
import pandas as pd
import numpy as np
from lightkurve import search_targetpixelfile
from astropy.coordinates import SkyCoord
import astropy.units as u
import os

# NGC 6819 membership list (all 30 stars from the paper)
ngc6819_members = pd.DataFrame({
    'KIC': [5111207, 5023899, 5024227, 5023760, 5024122, 5113601, 5112499, 5026583, 4938993, 5111834,
            5111908, 5024856, 5112507, 5024280, 5023796, 5024008, 5023724, 5023875, 5112268, 5111939,
            5025271, 4937169, 5112871, 5023666, 5024182, 5023926, 4937149, 4936891, 4937119, 4937356],
    'RA': [295.28458, 295.46213, 295.45978, 295.32355, 295.31873, 295.32724, 295.46078, 295s.21735, 
           295.27377, 295.28732, 295.31842, 295.45093, 295.38077, 295.32724, 295.48907, 295.18051, 
           295.42371, 295.35238, 295.32745, 295.30096, 295.30096, 295.42371, 295.35238, 295.32724,
           295.32745, 295.30096, 295.42371, 295.35238, 295.32724, 295.32745],
    'DEC': [40.21639, 40.18481, 40.19511, 40.20554, 40.21249, 40.21387, 40.20777, 40.20320, 
            40.20909, 40.17926, 40.18683, 40.22015, 40.21639, 40.24905, 40.18657, 40.26066, 
            40.19649, 40.20320, 40.18131, 40.18915, 40.18915, 40.19649, 40.20320, 40.21387,
            40.18131, 40.18915, 40.19649, 40.20320, 40.21387, 40.18131],
    'V': [14.787, 14.712, 14.718, 14.698, 14.892, 15.043, 15.102, 15.393, 15.678, 15.384,
          15.344, 15.671, 15.495, 15.738, 15.560, 15.784, 15.783, 15.880, 15.834, 15.945,
          15.965, 16.432, 16.516, 16.515, 16.604, 16.677, 16.929, 17.033, 17.134, 17.266]
})

def check_tess_data(ra, dec, kic):
    """Check if TESS data is available for a given RA and Dec."""
    coord = SkyCoord(ra=ra*u.degree, dec=dec*u.degree)
    tpf = search_targetpixelfile(coord, mission='TESS', radius=0.02*u.deg)
    
    if len(tpf) > 0:
        # TESS data available, save information
        tpf_data = tpf[0].download()
        header = tpf_data.get_header()
        info = {
            'KIC': kic,
            'TIC': header.get('TICID'),
            'Sector': header.get('SECTOR'),
            'Camera': header.get('CAMERA'),
            'CCD': header.get('CCD'),
            'Start_Time': tpf_data.time[0].value,
            'End_Time': tpf_data.time[-1].value
        }
        return True, info
    else:
        return False, None


total_stars = len(ngc6819_members)
tess_data_available = 0
tess_data_info = []

for _, star in ngc6819_members.iterrows():
    print(f"Checking TESS data for KIC {star['KIC']}...")
    has_data, info = check_tess_data(star['RA'], star['DEC'], star['KIC'])
    
    if has_data:
        tess_data_available += 1
        tess_data_info.append(info)
        print(f"TESS data found for KIC {star['KIC']}")
    else:
        print(f"No TESS data found for KIC {star['KIC']}")

print(f"\nTotal KIC stars: {total_stars}")
print(f"KIC stars with TESS data: {tess_data_available}")

# Save TESS data information to a CSV file
if tess_data_info:
    df = pd.DataFrame(tess_data_info)
    output_file = 'ngc6819_tess_data_info.csv'
    df.to_csv(output_file, index=False)
    print(f"TESS data information saved to {output_file}")

# Print details of stars with TESS data
print("\nDetails of NGC 6819 stars with TESS data:")
for info in tess_data_info:
    print(f"KIC: {info['KIC']}, TIC: {info['TIC']}, Sector: {info['Sector']}, Camera: {info['Camera']}, CCD: {info['CCD']}")

Checking TESS data for KIC 5111207.0...
No TESS data found for KIC 5111207.0
Checking TESS data for KIC 5023899.0...
No TESS data found for KIC 5023899.0
Checking TESS data for KIC 5024227.0...
No TESS data found for KIC 5024227.0
Checking TESS data for KIC 5023760.0...
TESS data found for KIC 5023760.0
Checking TESS data for KIC 5024122.0...
No TESS data found for KIC 5024122.0
Checking TESS data for KIC 5113601.0...
No TESS data found for KIC 5113601.0
Checking TESS data for KIC 5112499.0...
No TESS data found for KIC 5112499.0
Checking TESS data for KIC 5026583.0...
No TESS data found for KIC 5026583.0
Checking TESS data for KIC 4938993.0...
No TESS data found for KIC 4938993.0
Checking TESS data for KIC 5111834.0...
TESS data found for KIC 5111834.0
Checking TESS data for KIC 5111908.0...
TESS data found for KIC 5111908.0
Checking TESS data for KIC 5024856.0...
No TESS data found for KIC 5024856.0
Checking TESS data for KIC 5112507.0...
TESS data found for KIC 5112507.0
Checking TE

OSError: [Errno 30] Read-only file system: 'ngc6819_tess_data_info.csv'

In [31]:
tess_data_info

[{'KIC': 5023760.0,
  'TIC': 1880383370,
  'Sector': 41,
  'Camera': 1,
  'CCD': 2,
  'Start_Time': 2419.992552367399,
  'End_Time': 2446.5813433197213},
 {'KIC': 5111834.0,
  'TIC': 139153729,
  'Sector': 40,
  'Camera': 2,
  'CCD': 4,
  'Start_Time': 2390.655914963728,
  'End_Time': 2418.856451974003},
 {'KIC': 5111908.0,
  'TIC': 1880383370,
  'Sector': 41,
  'Camera': 1,
  'CCD': 2,
  'Start_Time': 2419.992552367399,
  'End_Time': 2446.5813433197213},
 {'KIC': 5112507.0,
  'TIC': 139154123,
  'Sector': 41,
  'Camera': 1,
  'CCD': 2,
  'Start_Time': 2419.9925501394428,
  'End_Time': 2446.5813430517333},
 {'KIC': 5024280.0,
  'TIC': 139154287,
  'Sector': 41,
  'Camera': 1,
  'CCD': 2,
  'Start_Time': 2419.992547756887,
  'End_Time': 2446.5813391851148},
 {'KIC': 5023724.0,
  'TIC': 184010831,
  'Sector': 41,
  'Camera': 1,
  'CCD': 2,
  'Start_Time': 2419.9925525660037,
  'End_Time': 2446.581347003335},
 {'KIC': 5023875.0,
  'TIC': 139154123,
  'Sector': 41,
  'Camera': 1,
  'CCD': 

In [26]:
from astroquery.nasa_exoplanet_archive import NasaExoplanetArchive
import pandas as pd

def fetch_kepler_stars():
    query = """
    SELECT kepid, teff, radius, mass, logg, feh, kic_kepmag
    FROM keplerstellar
    WHERE teff BETWEEN 5600 AND 6000
      AND radius BETWEEN 0.9 AND 1.1
      AND nu_max IS NULL
    ORDER BY teff
    LIMIT 10
    """
    
    try:
        results = NasaExoplanetArchive.query_criteria(table="keplerstellar", query=query)
        
        if len(results) > 0:
            df = results.to_pandas()
            return df
        else:
            print("No results found matching the criteria.")
            return None
    except Exception as e:
        print(f"An error occurred: {e}")
        return None

def main():
    stars_data = fetch_kepler_stars()
    if stars_data is not None and not stars_data.empty:
        print(stars_data.to_string(index=False))
    else:
        print("Failed to fetch star data.")

if __name__ == "__main__":
    main()

KeyboardInterrupt: 